In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '2b03b0d1e29b48f48ac266ad0a85d505'
CLIENT_SECRET = '79aefa91afc841fbb1726f4d6dbba454'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 7.6 MB/s eta 0:00:00


In [5]:
def get_trending_playlist_data(playlist_id, access_token):
    """
    Fetches data for tracks in a given playlist using Spotify's API.

    Args:
        playlist_id (str): The ID of the playlist.
        access_token (str): The access token for authenticating with Spotify's API.

    Returns:
        pandas.DataFrame: DataFrame containing information about the tracks in the playlist.
    """
    try:
        # Set up Spotipy with the access token
        sp = spotipy.Spotify(auth=access_token)

        # Get the tracks from the playlist
        playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')['items']

        # Extract relevant information and store in a list of dictionaries
        music_data = []
        for track_info in playlist_tracks:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            # Get audio features for the track
            audio_features = sp.audio_features(track_id)[0] if track_id else None

            # Get release date of the album
            release_date = sp.album(album_id)['release_date'] if album_id else None

            # Get popularity of the track
            popularity = sp.track(track_id)['popularity'] if track_id else None

            # Add additional track information to the track data
            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': sp.track(track_id).get('explicit', None),
                'External URLs': sp.track(track_id).get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
                # Add more attributes as needed
            }

            music_data.append(track_data)

        # Create a pandas DataFrame from the list of dictionaries
        df = pd.DataFrame(music_data)

        return df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [28]:
playlist_id = 'https://open.spotify.com/playlist/37i9dQZF1DWWOp144dTCQd?si=f7c14d13d38f42d5'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                   Track Name  \
0                  Valaiyosai   
1             Sangeetha Megam   
2    Malaiyoram Veesum Kaathu   
3            Sorgame Endralum   
4       Thenpandi Thamizhe En   
..                        ...   
95            Madai Thiranthu   
96  Andhi Mazhai Pozhikaruthu   
97          Ithu Oru Ponmalai   
98       En Jodi Manja Kuruvi   
99            Thendral Vandhu   

                                              Artists  \
0   Ilaiyaraaja, S. P. Balasubrahmanyam, Lata Mang...   
1                              S. P. Balasubrahmanyam   
2                              S. P. Balasubrahmanyam   
3                              Ilaiyaraaja, S. Janaki   
4                        K. J. Yesudas, K. S. Chithra   
..                                                ...   
95                             S. P. Balasubrahmanyam   
96                  S. P. Balasubrahmanyam, S. Janaki   
97                             S. P. Balasubrahmanyam   
98  S. P. Balasubrahmanyam, S. P.

In [29]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [31]:
 #Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [32]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [33]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [34]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Create a DataFrame with the input song data
    input_song_data = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    # Concatenate content-based recommendations with the input song data
    hybrid_recommendations = pd.concat([content_based_rec, input_song_data], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations


In [36]:
input_song_name = "Valaiyosai"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Valaiyosai':
                 Track Name  \
0             Unna Nenachen   
3  Thenmadurai Vaigai Nadhi   
4                Indha Maan   
1         Ithu Oru Ponmalai   
2     Un Nenja Thottu Sollu   

                                             Artists  \
0                             S. P. Balasubrahmanyam   
3  S. P. Balasubrahmanyam, P. Susheela, Malaysia ...   
4                         Ilaiyaraaja, K. S. Chithra   
1                             S. P. Balasubrahmanyam   
2                         P. Susheela, K. S. Chithra   

                                          Album Name Release Date  Popularity  
0  Apoorva Sagodharargal (Original Motion Picture...   1989-04-14        50.0  
3  Dharmathin Thalaivan (Original Motion Picture ...         1988        50.0  
4  Karakattakkaran (Original Motion Picture Sound...   1989-06-16        48.0  
1                                          Nizhalgal   2005-06-30        45.0  
2  Rajathi Raja (Original Motion 